# Preprocess module

## Import Jupyter Module in other files

this code is for importing .ipynb files. YEYEEE

- code form <br/ > 
http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

In [1]:
import io, os, sys, types
from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

In [2]:
def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [3]:
class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod

In [4]:
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

- word2vec 모듈 다운로드가 요구됨

In [6]:
import format_module
import bookingreview
import preprocessreview
import wordvectormaker
import postag_module
import gensim
import xl_to_br_module
#sys.setrecursionlimit(20000)

C:\Users\Shako\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# 데이터 저장

In [7]:
import _pickle
import gc

def save_object_split(obj, filename, ext, split_size = 1000):
    index = 0
    for i in range(0, len(obj), split_size):
        with open(filename + "_p" + str(index) + "." + ext, 'wb') as output:
            _pickle.dump(obj[i:i+split_size], output)
            index += 1
            mem()
            gc.collect()
                
def save_object(obj, filename):
    with open(filename, 'wb') as output:
        _pickle.dump(obj, output)

def load_object(filename):
    with open(filename, 'rb') as input:
        return _pickle.load(input)
    
def load_object_split(filename, ext, last_index):
    for index in range(last_index+1):
        with open(filename + "_p" + str(index) + "." + ext, 'rb') as input:
            if(index == 0):
                out = _pickle.load(input)
            else:
                out.extend(_pickle.load(input))
    
    return out

# Main

In [8]:
def postagModule(xl_name, reviewDB, mode = "komoran"):
    BRList = xl_to_br_module.xl_to_BookingReview(xl_name)
    
    print("adding review to DB...")
    reviewDB.add_review_list(BRList)
    
    PRList = postag_module.komoran(BRList)
    
    '''save_object(PRList_twitter, "save_PR_twitter.pkl")
    save_object(PRList_komoran, "save_PR_komoran.pkl")'''
    
    return PRList

In [9]:
def embeddingModule(PRList):
    word2vec_model = gensim.models.Word2Vec.load('./models/namuwiki_testmodel_Komoran.model')
    print("processing word embedding...")
    return wordvectormaker.word2vec_to_PreprocessReview_list(word2vec_model, PRList)

In [10]:
def formattingModule(PRList, reviewDB, version):
    print("processing formatted review...")
    formatted_list = [format_module.FormattedReview(review) for review in PRList]
    print("saving formatted review...")
    save_object(formatted_list, "./pkl/save_formatted_review_" + version +".pkl")
    return formatted_list

In [11]:
def preprocessModule(xl_name, reviewDB, version):
    PRList = postagModule(xl_name, reviewDB)
    embeddingModule(PRList)
    formatted_list = formattingModule(PRList, reviewDB, version)
    
    return formatted_list

In [12]:
reviewDB = format_module.ReviewDB("train")
format_module.FormattedReview.setDB(reviewDB)
formatted_list = preprocessModule("Commonreviews_snuproject _test.xlsx", reviewDB, "test")

processing 0 th unit...
adding review to DB...
=> Postag initiated
postagging [komoran] 0 ith unit...
processing word embedding...

stemmed :  ['웃다']
postagged : ['웃/VV', '다/EC']
웃으/NA not in vocabulary so replaced to ['웃/VV', '다/EC']

stemmed :  ['좋다']
postagged : ['좋/VA', '다/EC']
좋겟/NA not in vocabulary so replaced to ['좋/VA', '다/EC']

stemmed :  ['식신']
postagged : ['식신/NNP']
식신 로드/NNP not in vocabulary so replaced to ['식신/NNP']

stemmed :  ['없네욬']
postagged : ['없네욬/NA']
없네욬/NA not in vocabulary so replaced to []

stemmed :  ['도', '그렇다']
postagged : ['도/NNG', '그렇/VA', '다/EC']
도그렇겟죠/NA not in vocabulary so replaced to ['도/NNG', '그렇/VA', '다/EC']

stemmed :  ['재밌다']
postagged : ['재밌/VA', '다/EC']
재밌/NA not in vocabulary so replaced to ['재밌/VA', '다/EC']

stemmed :  ['아깝다']
postagged : ['아깝/VA', '다/EC']
아깝/NA not in vocabulary so replaced to ['아깝/VA', '다/EC']

stemmed :  ['스럽다']
postagged : ['스럽/XSA', '다/EC']
스럽/NA not in vocabulary so replaced to ['스럽/XSA', '다/EC']

stemmed :  ['바라다']
pos

postagged : ['여자/NNG']
여자 친구/NNP not in vocabulary so replaced to ['여자/NNG']

stemmed :  ['강츄']
postagged : ['강츄/NA']
강츄/NA not in vocabulary so replaced to []

stemmed :  ['나쁘다']
postagged : ['나쁘/VA', '다/EC']
나쁘/NA not in vocabulary so replaced to ['나쁘/VA', '다/EC']

stemmed :  ['퀄러']
postagged : ['퀄러/NA']
퀄러/NA not in vocabulary so replaced to []

stemmed :  ['아깝다']
postagged : ['아깝/VA', '다/EC']
아깝/NA not in vocabulary so replaced to ['아깝/VA', '다/EC']

stemmed :  ['바쁘다']
postagged : ['바쁘/VA', '다/EC']
바쁘/NA not in vocabulary so replaced to ['바쁘/VA', '다/EC']

stemmed :  ['아니다']
postagged : ['아니/VCN', '다/EC']
아녔구/NA not in vocabulary so replaced to ['아니/VCN', '다/EC']

stemmed :  ['렸', '다']
postagged : ['렸/NA', '다/MAG']
렸다/NA not in vocabulary so replaced to ['렸/NA', '다/MAG']

stemmed :  ['좋다', '더']
postagged : ['좋/VA', '다/EC', '더/MAG']
좋더/NA not in vocabulary so replaced to ['좋/VA', '다/EC', '더/MAG']

stemmed :  ['재밌다']
postagged : ['재밌/VA', '다/EC']
재밌/NA not in vocabulary so replaced to 


stemmed :  ['같슴돠']
postagged : ['같슴돠/NA']
같슴돠/NA not in vocabulary so replaced to []

stemmed :  ['재밌다']
postagged : ['재밌/VA', '다/EC']
재밌/NA not in vocabulary so replaced to ['재밌/VA', '다/EC']

stemmed :  ['많다']
postagged : ['많/VA', '다/EC']
많더/NA not in vocabulary so replaced to ['많/VA', '다/EC']

stemmed :  ['스럽다']
postagged : ['스럽/XSA', '다/EC']
스럽/NA not in vocabulary so replaced to ['스럽/XSA', '다/EC']

stemmed :  ['침밷을까바']
postagged : ['침밷을까바/NA']
침밷을까바/NA not in vocabulary so replaced to []

stemmed :  ['아깝다']
postagged : ['아깝/VA', '다/EC']
아깝/NA not in vocabulary so replaced to ['아깝/VA', '다/EC']

stemmed :  ['더브릿지']
postagged : ['더브릿지/NNP']
더브릿지/NNP not in vocabulary so replaced to []

stemmed :  ['쁠레']
postagged : ['쁠레/NA']
쁠레/NA not in vocabulary so replaced to []

stemmed :  ['찌다', '하다']
postagged : ['찌/VV', '다/EC', '하/VV', '다/EC']
찝했구/NA not in vocabulary so replaced to ['찌/VV', '다/EC', '하/VV', '다/EC']

stemmed :  ['가페']
postagged : ['가페/NA']
가페/NA not in vocabulary so replaced t

In [ ]:
#reviewDB = format_module.ReviewDB("validation")
reviewDB = format_module.ReviewDB("train")
print(reviewDB.review_dict[41099].next)

In [ ]:
a = load_object("./pkl/save_formatted_review_train.pkl")
print(a[0])

In [ ]:
print(formatted_list[0].context.shape)
print(formatted_list[1].context.shape)
print(formatted_list[2].context.shape)

In [ ]:
import bookingreview
import format_module

reviewDB = format_module.ReviewDB()
reviewList = [bookingreview.BookingReview("회사", "id", 10, "asdfsadf", 40000.236+i*0.001, True, i) for i in range(50000)]

reviewDB.add_review_list(reviewList)
reviewDB.size()